In [1]:
%sh
/databricks/python/bin/pip install --upgrade pip

Requirement already up-to-date: pip in /databricks/python3/lib/python3.5/site-packages (19.0.3)

In [2]:
%sh
/databricks/python/bin/pip install pmdarima

Requirement already satisfied: pmdarima in /databricks/python3/lib/python3.5/site-packages (1.1.0)
Requirement already satisfied: numpy>=1.10 in /databricks/python3/lib/python3.5/site-packages (from pmdarima) (1.11.1)
Requirement already satisfied: Cython>=0.29 in /databricks/python3/lib/python3.5/site-packages (from pmdarima) (0.29.6)
Requirement already satisfied: pandas>=0.19 in /databricks/python3/lib/python3.5/site-packages (from pmdarima) (0.24.2)
Requirement already satisfied: scipy>=1.0 in /databricks/python3/lib/python3.5/site-packages (from pmdarima) (1.2.1)
Requirement already satisfied: scikit-learn>=0.19 in /databricks/python3/lib/python3.5/site-packages (from pmdarima) (0.20.3)
Requirement already satisfied: statsmodels>=0.9.0 in /databricks/python3/lib/python3.5/site-packages (from pmdarima) (0.9.0)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.5/site-packages (from pandas>=0.19->pmdarima) (2016.6.1)
Requirement already satisfied: python-dateutil>=2.5.0 in /databricks/python3/lib/python3.5/site-packages (from pandas>=0.19->pmdarima) (2.5.3)
Requirement already satisfied: patsy in /databricks/python3/lib/python3.5/site-packages (from statsmodels>=0.9.0->pmdarima) (0.4.1)
Requirement already satisfied: six>=1.5 in /usr/lib/python3/dist-packages (from python-dateutil>=2.5.0->pandas>=0.19->pmdarima) (1.10.0)

In [3]:
dbutils.library.installPyPI("numpy","1.15.4")
dbutils.library.restartPython()

In [4]:
dbutils.library.installPyPI("pmdarima","1.1.0")
dbutils.library.restartPython()

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pmdarima.arima import auto_arima
from pmdarima.arima.stationarity import ADFTest
from sklearn.metrics import mean_squared_error
from math import sqrt

In [6]:
df = pd.read_csv("/dbfs/FileStore/tables/AAPL_complete.csv", parse_dates=['Date'], index_col=0)
df.head()

Out[ 2 ]: 
 Open High Low Close Adj Close Volume
Date 
1980-12-12 0.513393 0.515625 0.513393 0.513393 0.023007 117258400
1980-12-15 0.488839 0.488839 0.486607 0.486607 0.021807 43971200
1980-12-16 0.453125 0.453125 0.450893 0.450893 0.020206 26432000
1980-12-17 0.462054 0.464286 0.462054 0.462054 0.020706 21610400
1980-12-18 0.475446 0.477679 0.475446 0.475446 0.021307 18362400

In [7]:
df[160:170]

Out[ 3 ]: 
 Open High Low Close Adj Close Volume
Date 
1981-03-08 0.446429 0.446429 0.441964 0.441964 0.019806 3108000
1981-04-08 0.448661 0.450893 0.448661 0.448661 0.020106 7918400
1981-05-08 0.462054 0.464286 0.462054 0.462054 0.020706 4373600
1981-06-08 0.453125 0.453125 0.450893 0.450893 0.020206 2632000
1981-07-08 0.450893 0.453125 0.450893 0.450893 0.020206 2301600
1981-11-08 0.441964 0.441964 0.437500 0.437500 0.019606 17864000
1981-12-08 0.430804 0.430804 0.428571 0.428571 0.019206 6568800
1981-08-13 0.417411 0.417411 0.415179 0.415179 0.018606 6871200
1981-08-14 0.412946 0.412946 0.408482 0.408482 0.018306 6048000
1981-08-17 0.399554 0.399554 0.395089 0.395089 0.017705 4726400

In [8]:
df = df.drop(['Open','High','Low','Adj Close','Volume'], axis=1)
df.head()

Out[ 4 ]: 
 Close
Date 
1980-12-12 0.513393
1980-12-15 0.486607
1980-12-16 0.450893
1980-12-17 0.462054
1980-12-18 0.475446

In [9]:
stepwise_model = auto_arima(df, start_p=1, start_q=1, max_p=3, max_q=3, m=12, start_P=0, seasonal=False, d=1, D=1, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

Fit ARIMA: order=(1, 1, 1); AIC=24203.171, BIC=24231.847, Fit time=1.767 seconds
Fit ARIMA: order=(0, 1, 0); AIC=24213.540, BIC=24227.877, Fit time=0.005 seconds
Fit ARIMA: order=(1, 1, 0); AIC=24206.834, BIC=24228.341, Fit time=0.135 seconds
Fit ARIMA: order=(0, 1, 1); AIC=24206.070, BIC=24227.577, Fit time=0.098 seconds
Fit ARIMA: order=(2, 1, 1); AIC=24191.852, BIC=24227.696, Fit time=1.599 seconds
Fit ARIMA: order=(2, 1, 0); AIC=24192.157, BIC=24220.832, Fit time=0.143 seconds
Fit ARIMA: order=(2, 1, 2); AIC=24167.430, BIC=24210.442, Fit time=4.653 seconds
Fit ARIMA: order=(3, 1, 3); AIC=24102.772, BIC=24160.122, Fit time=12.571 seconds
Fit ARIMA: order=(2, 1, 3); AIC=24104.970, BIC=24155.151, Fit time=8.496 seconds
Fit ARIMA: order=(3, 1, 2); AIC=24105.139, BIC=24155.321, Fit time=11.714 seconds
Total fit time: 41.187 seconds

In [10]:
print(stepwise_model.aic())

24102.772092715313

In [11]:
train = df.loc['1980-12-12':'2010-12-31']
test = df.loc['2011-01-03':]

In [12]:
stepwise_model.fit(train)

Out[ 8 ]: 
ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(3, 1, 3),
 out_of_sample_size=0, scoring='mse', scoring_args={},
 seasonal_order=None, solver='lbfgs', start_params=None,
 suppress_warnings=True, transparams=True, trend=None,
 with_intercept=True)

In [13]:
def smape(A, F):
    return 1/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [14]:
def statResult(n_periods):
    future_forecast = stepwise_model.predict(n_periods)

    rmse = sqrt(mean_squared_error(future_forecast, test[:n_periods]['Close'].tolist()))
    print("RMSE")
    print(rmse)
    print("SMAPE")
    print(smape(test[:n_periods]['Close'].tolist(),future_forecast))

In [15]:
statResult(10)


RMSE
4.398347119057009
SMAPE
0.09013564462428648